# 线性神经网络

## 3.1 线性回归
<div align=center>
<img src=./img/chapter03/1.jpg >
</div>

In [10]:
import torch
import random
def synthetic_data(w,b,num_examples):
    x=torch.normal(0,1,(num_examples,len(w)))
    y=torch.matmul(x,w)+b
    y+=torch.normal(0,0.01,y.shape)
    return x,y.reshape((-1,1))

def data_iter(batch_size, features, labels): 
    num_examples = len(features) 
    indices = list(range(num_examples)) 
    # 这些样本是随机读取的，没有特定的顺序 
    random.shuffle(indices) 
    for i in range(0, num_examples, batch_size): 
        batch_indices = torch.tensor( indices[i: min(i + batch_size, num_examples)]) 
        yield features[batch_indices], labels[batch_indices]

true_w=torch.tensor([2,-3.4])
true_b=torch.tensor(4.2)

def liner_net(X,w,b):
    return torch.matmul(X,w)+b
def loss(y_hat,y):
    return(y_hat-y.reshape(y_hat.shape))**2/2
def sgd(params,lr,batchszie):
    with torch.no_grad():
        for param in params:
            param-=lr*param.grad/batchszie
            param.grad.zero_()
def train():
    lr=0.01
    num_epochs=10
    w = torch.normal(0, 0.01, size=(2,1), requires_grad=True) 
    b = torch.zeros(1, requires_grad=True)
    batchsize=10
    feature,labels=synthetic_data(true_w,true_b,1000)
    for epoch in range(num_epochs):
        for X,y in data_iter(batchsize,feature,labels):
            l=loss(liner_net(X,w,b),y)
            l.sum().backward()
            sgd([w,b],lr,batchsize)
        print(f'epoch{epoch+1},loss{loss(net(feature,w,b),labels).mean():f}')

train()

epoch1,loss2.178056
epoch2,loss0.282730
epoch3,loss0.036742
epoch4,loss0.004823
epoch5,loss0.000672
epoch6,loss0.000129
epoch7,loss0.000058
epoch8,loss0.000049
epoch9,loss0.000048
epoch10,loss0.000048


## 3.2 logstic回归

## 3.3 softmax回归

+ softmax回归和线性回归有什么区别？

线性回归是解决预测问题，输出空间是连续的数值。而softmax回归是分类问题，输出空间是离散的概率值。

### 基础概念
1. 独热编码(one-hot coding)：独热编码是一个向量，它的分量和类别一样多。类别相应的分量为1，其他分量为0。例如：(1; 0; 0)对应于 “猫”、(0; 1; 0)对应于“鸡”、(0; 0; 1)对应于“狗。
2. softmax函数：$\hat{y_j}=\frac{exp(o_j)}{\sum_k exp(o_j)}$
3. 尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。因此，softmax回 归是一个线性模型。

<img src=./img/chapter03/2.jpg>

In [11]:
%matplotlib inline
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l


In [12]:
# 加载数据集
def load_data(batchsize,resize=None):
    trans=[transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans=transforms.Compose(trans)
    mnist_train=torchvision.datasets.FashionMNIST(
        root="./data",train=True,transform=trans,download=False
    )
    mnist_test=torchvision.datasets.FashionMNIST(
        root="./data",train=False,transform=trans,download=False
    )
    return (data.DataLoader(mnist_train, batchsize, shuffle=True, num_workers=1), 
            data.DataLoader(mnist_test, batchsize, shuffle=False, num_workers=1))


In [13]:
class Accumulator: #@save """在`n`个变量上累加。""" 
    def __init__(self, n): 
        self.data = [0.0] * n 
    def add(self, *args): 
        self.data = [a + float(b) for a, b in zip(self.data, args)] 
    def reset(self): 
        self.data = [0.0] * len(self.data) 
    def __getitem__(self, idx):
        return self.data[idx]

In [18]:
def softmax(X):
    X_exp=torch.exp(X)
    partition=X_exp.sum(1,keepdim=True)
    return X_exp/partition #广播机制
def softmax_net(X,w,b):
    return softmax(torch.matmul(X.reshape((-1,w.shape[0])),w)+b)
def cross_entroy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])
def update(w,b,lr,batchsize):
    return d2l.sgd([w,b],lr,batchsize)
def accuracy(y_hat,y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1: 
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y 
    return float(cmp.type(y.dtype).sum())
def evaluate_accuracy(w,b,data_iter):
    metric=Accumulator(2)
    for x,y in data_iter:
        metric.add(accuracy(net(x,w,b),y),y.numel())
    return metric[0]/metric[1]


In [ ]:
def train():
    num_inputs=784
    num_outputs=10
    W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True) 
    b = torch.zeros(num_outputs, requires_grad=True)
    batchsize=10
    num_epochs=10
    train_iter,test_iter=load_data(batchsize=batchsize)
    for epoch in range(num_epochs):
        for x,y in train_iter:
            y_hat=softmax_net(x,W,b)
            l=cross_entroy(y_hat,y)
            l.sum().backward()
            update(W,b,lr=0.01,batchsize=batchsize)
        test_acc=evaluate_accuracy(W,b,test_iter)
        print(f"epoch:{epoch+1},test_acc:{test_acc:f}")
train()